In [1]:
import numpy as np
import pandas as pd 

In [2]:
df = pd.read_csv('covid_toy.csv')

In [3]:
df.head()

age  gender  fever cough     city has_covid
0   60    Male  103.0  Mild  Kolkata        No
1   27    Male  100.0  Mild    Delhi       Yes
2   42    Male  101.0  Mild    Delhi        No
3   31  Female   98.0  Mild  Kolkata        No
4   65  Female  101.0  Mild   Mumbai        No

In [4]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [5]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(df.drop(columns=['has_covid']),df['has_covid'],
                                                test_size=0.2)

In [6]:
X_train

age  gender  fever   cough       city
7    20  Female    NaN  Strong     Mumbai
33   26  Female   98.0    Mild    Kolkata
65   69  Female  102.0    Mild  Bangalore
20   12    Male   98.0  Strong  Bangalore
74   34  Female  104.0  Strong      Delhi
..  ...     ...    ...     ...        ...
51   11  Female  100.0  Strong    Kolkata
95   12  Female  104.0    Mild  Bangalore
50   19    Male  101.0    Mild      Delhi
84   69  Female   98.0  Strong     Mumbai
64   42    Male  104.0    Mild     Mumbai

[80 rows x 5 columns]

## 1. Aam Zindagi (Without Columns Transformer)

**adding simple imputer to fever col**

In [39]:
from sklearn.impute import SimpleImputer
si = SimpleImputer()
X_train_fever = si.fit_transform(X_train[['fever']])

# also the test data
X_test_fever = si.fit_transform(X_test[['fever']])
                                 
X_train_fever.shape

(80, 1)

**Ordinalencoding -> cough**

In [40]:
from sklearn.preprocessing import OrdinalEncoder
oe = OrdinalEncoder(categories=[['Mild','Strong']])
X_train_cough = oe.fit_transform(X_train[['cough']])

# also the test data
X_test_cough = oe.fit_transform(X_test[['cough']])

X_train_cough.shape

(80, 1)

**OneHotEncoding -> gender,city**

In [41]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(drop='first',sparse=False)
X_train_gender_city = ohe.fit_transform(X_train[['gender','city']])

# also the test data
X_test_gender_city = ohe.fit_transform(X_test[['gender','city']])

X_train_gender_city.shape

(80, 4)

In [42]:
# Extracting Age
X_train_age = X_train.drop(columns=['gender','fever','cough','city']).values

# also the test data
X_test_age = X_test.drop(columns=['gender','fever','cough','city']).values

X_train_age.shape

(80, 1)

here we can simply extract age but it will not give us multidimensional array

In [43]:
X_train_transformed = np.concatenate((X_train_age,X_train_fever,X_train_gender_city,X_train_cough),axis=1)
# also the test data
X_test_transformed = np.concatenate((X_test_age,X_test_fever,X_test_gender_city,X_test_cough),axis=1)

X_train_transformed.shape

(80, 7)

## Using ColumnTransformer (Mentos Jindgi)

In [44]:
from sklearn.compose import ColumnTransformer

In [47]:
transformer = ColumnTransformer(transformers =[
    ('tnf1' , SimpleImputer() , ['fever']) , 
    ('tnf2',OrdinalEncoder(categories=[['Mild','Strong']]),['cough']),
    ('tnf3' , OneHotEncoder(sparse = False , drop = 'first') , ['gender' , 'city'])
    
] , remainder = 'passthrough')

In [48]:
transformer.fit_transform(X_train).shape

(80, 7)

In [49]:
transformer.transform(X_test).shape

(20, 7)